<a href="https://colab.research.google.com/github/mccoymb/AAE-590-DSMM/blob/main/590DSMM_HW6_3_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pymc as pm
import pandas as pd
import numpy as np
import arviz as az
from google.colab import files
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import mean_squared_error, r2_score

# Upload the Excel file
uploaded = files.upload()

# Load data
df = pd.read_excel("GeneratedExcel.xlsx")
X = df.iloc[:, 0].values.reshape(-1, 1)  # First column: independent variable
dependent_vars = df.columns[1:13]        # Columns B-M: dependent variables

Saving GeneratedExcel.xlsx to GeneratedExcel.xlsx


In [5]:
results = []

for col_index in range(1, 13):
    y = df.iloc[:, col_index].values
    y_col = df.columns[col_index]

    # BLR
    with pm.Model() as model:
        alpha = pm.Normal("alpha", mu=213.8, sigma=1)
        beta = pm.Normal("beta", mu=200, sigma=50)
        sigma = pm.HalfNormal("sigma", sigma=1)
        mu = alpha + beta * X.flatten()
        y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)
        trace = pm.sample(2000, return_inferencedata=True, progressbar=False, cores=2)

    alpha_post = trace.posterior["alpha"].mean().item()
    beta_post = trace.posterior["beta"].mean().item()
    y_pred_blr = alpha_post + beta_post * X.flatten()

    r2_blr = r2_score(y, y_pred_blr)
    rmse_blr = np.sqrt(mean_squared_error(y, y_pred_blr))
    # GPR
    kernel = C(1, (1e-3, 1e4)) * RBF(1.0, (1e-2, 1e2))
    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
    gpr.fit(X, y)
    y_gpr_eval, _ = gpr.predict(X, return_std=True)

    r2_gpr = r2_score(y, y_gpr_eval)
    rmse_gpr = np.sqrt(mean_squared_error(y, y_gpr_eval))
    results.append({
        'Target': y_col,
        'BLR R²': f"{r2_blr:.4f}",
        'BLR RMSE': f"{rmse_blr:.4f}",
        'GPR R²': f"{r2_gpr:.4f}",
        'GPR RMSE': f"{rmse_gpr:.4f}"
    })

# ----- Tabular Output -----
results_df = pd.DataFrame(results)
print("\n📋 Copy & Paste the table below into Word (tab-separated, use Consolas font):\n")
print(results_df.to_csv(sep='\t', index=False))

results_df.to_csv("blr_gpr_results.csv", index=False)


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing


📋 Copy & Paste the table below into Word (tab-separated, use Consolas font):

Target	BLR R²	BLR RMSE	GPR R²	GPR RMSE
Holes 1mm	-0.1447	204.1121	1.0000	0.0000
Mat A 1mm	0.4599	61.6997	1.0000	0.0000
Mat B 1mm	0.8821	6.1288	1.0000	0.0000
Holes 5mm	-1.0223	1354.0380	1.0000	0.0000
Mat A 5mm	-0.6196	532.5005	1.0000	0.0000
Mat B 5mm	-16.1580	369.4289	1.0000	0.0000
Holes 100N	-194.5482	93.2077	1.0000	0.0000
Mat A 100N	-101808.0350	90.5178	1.0000	0.0000
Mat B 100N	-2797416.9793	90.4210	1.0000	0.0000
Holes 500N	-8.9844	105.3157	1.0000	0.0000
Mat A 500N	-4000.1589	89.7142	1.0000	0.0000
Mat B 500N	-108934.4928	89.2355	1.0000	0.0000

